This notebook works through the process of building mineral element and commodity information into the GeoKB. We need these concepts, associated identifiers linking them to other systems, and some specific characteristics. At this stage, all information associated with minerals comes from external sources to be integrated into the GeoKB here. We do not yet have any use cases pointing toward the GeoKB needing to house characteristics that we can't effectively source from elsewhere.

Ultimately, MinDat likely provides the most workable community source for comprehensive information on minerals. The MinDat API is still under development, but we are experimenting with how it works and what we can get that we really need in our system.

In the meantime as a start, we are working with the listing of chemical elements from the USGS Thesaurus, a small subset that concentrates on those that show up in our assessment work and products. We also tie in to the list of commodities (some of which are mineral elements) from the MRDS code list. Bringing the identifiers for these into the GeoKB is necessary anyway as we need to link to them by ID from other sources, and they are a decent enough set for us to get started with other work.

In [45]:
import os
import pandas as pd
import requests
from zipfile import ZipFile
from io import BytesIO
import sqlite3

from utils import sparql_query, property_query

from wikibaseintegrator.wbi_config import config as wbi_config
from wikibaseintegrator import WikibaseIntegrator, wbi_login, wbi_helpers
from wikibaseintegrator.datatypes import Item, String, ExternalID, URL


In [87]:
# Vist the README for information on these variables
wbi_config['MEDIAWIKI_API_URL'] = os.environ['MEDIAWIKI_API_URL']
wbi_config['SPARQL_ENDPOINT_URL'] = os.environ['SPARQL_ENDPOINT_URL']
wbi_config['WIKIBASE_URL'] = os.environ['WIKIBASE_URL']

# Use bot account for this specific task
geokb_auth = wbi_login.Login(
    user=os.environ['WB_BOT_INIT'], 
    password=os.environ['WB_PASS_INIT']
)
wbi = WikibaseIntegrator(login=geokb_auth)

In [ ]:
def add_item(label: str, description: str, aliases: list|str|None, claims: list|None):
    item = wbi.item.new()

    item.labels.set(language='en', value=label)
    item.descriptions.set(language='en', value=description)
    item.aliases.set(language='en', values=aliases)
    item.claims.add(claims)

    response = item.write()

    return response



# Information Sources

The USGS reference source for mineral names (as chemical elements) comes from the USGS Thesaurus. This is a very limited set of records focused solely on those minerals for which USGS has a significant research or assessment focus and that turn up in our various data systems such as MRDS. We will ultimately establish a connection with MinDat and potentially other sources to incorporate more mineral items and additional details, but we can start here, with a need to incorporate the relevant internal identifiers specific to our own data that we will need for linking purposes.

We also have a set of commodity names and codes as part of MRDS, some of which are mineral elements and some of which are other types of commodities. This does not yet consider a few other commodities that are examined as part of NMIC operations, and we'll look at those in another future step.

In [11]:
# Download and extract thesaurus DB
# Need to rework this for in memory process if we set this process up as a lambda

usgs_thesaurus_url = 'https://apps.usgs.gov/thesaurus/download/thesauri.zip'
mrds_commodities_url = 'https://mrdata.usgs.gov/mrds/commodity-codes.txt'

thesaurus_fn = 'thesauri.db'

chem_elem_id = 1427
chem_elem_group_id = 1426

def get_thesaurus(fn=thesaurus_fn, url=usgs_thesaurus_url):
    if not os.path.isfile(fn):
        usgs_thesaurus_zip = requests.get(url)
        usgs_thesaurus_file = ZipFile(BytesIO(usgs_thesaurus_zip.content))
        usgs_thesaurus_file.extract(fn)
    
    return sqlite3.connect(fn)    

In [12]:
# All thesaurus terms we will work with are in one table, it's small enough we can grab everything
thesaurus_terms = pd.read_sql_query("SELECT * FROM term", get_thesaurus())

# We need to exploit term relationships so we build a table to include labels
related_terms = pd.read_sql_query("SELECT * FROM relterm", get_thesaurus())
related_terms_lookup = pd.merge(
    left=pd.merge(
        left=related_terms,
        right=thesaurus_terms[["code","name"]].rename(columns={'code': 'a', 'name': 'a_name'}),
        how="left",
        on='a'
    ),
    right=thesaurus_terms[["code","name"]].rename(columns={'code': 'b', 'name': 'b_name'}),
    how="left",
    on="b"
)

# Tease out two logical tables of terms we need as references - chemical elements and chemical element groups
chem_elem_groups = thesaurus_terms[thesaurus_terms.parent == chem_elem_group_id].copy()
chemical_elements = thesaurus_terms[thesaurus_terms.parent == chem_elem_id].copy()
chemical_elements["code_str"] = chemical_elements.code.apply(str)

# Wikidata Connection

Wikidata has an ExternalID for the USGS thesaurus containing the "tcode" value. Most of the chemical elements in the USGS Thesaurus link to Wikidata items for those elements, and establishing a linkage in the GeoKB means we will be able to exploit Wikidata information further as needed and provide a linkage from our system for new knowledge we organize and publish. Wikidata is perhaps not the best source for information on chemical elements, and we will need to be selective on what we pull from that source vs. the more professionally maintained MinDat.

In [13]:
# Get all Wikidata items that currently have a USGS Thesaurus tcode including those ExternalID property values
tcode_query = """
SELECT ?item ?itemLabel ?tcode ?chem_symbol
WHERE 
{
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  ?item wdt:P11095 [].
  OPTIONAL {
  ?item wdt:P11095 ?tcode.
  ?item wdt:P246 ?chem_symbol.
  }
}
"""

wd_items_with_usgs_tcode = sparql_query(
    endpoint="https://query.wikidata.org/sparql",
    query=tcode_query,
    output='dataframe'
)

In [15]:
# Merge the Wikidata identifiers along with chemical symbol values
chem_elems_wd_mapping = pd.merge(
    left=chemical_elements,
    right=wd_items_with_usgs_tcode,
    how="left",
    left_on="code_str",
    right_on="tcode"
)

# Look at a few cases where the labels don't match between the USGS Thesaurus and what claims to be an associated record in Wikidata
chem_elems_wd_mapping[chem_elems_wd_mapping['name'] != chem_elems_wd_mapping.itemLabel]

,code,name,parent,scope,code_str,item,itemLabel,tcode,chem_symbol
1,1390,aluminum,1427.0,Metalloid element with symbol Al and atomic nu...,1390,http://www.wikidata.org/entity/Q663,aluminium,1390,Al
19,1424,cesium,1427.0,Alkali metal element with symbol Cs and atomic...,1424,http://www.wikidata.org/entity/Q1108,caesium,1424,Cs
38,1466,hafnium,1427.0,Metal element with symbol Hf and atomic number...,1466,NaN,NaN,NaN,NaN
71,1539,plutonium,1427.0,Rare earth (actinide series) element with symb...,1539,http://www.wikidata.org/entity/Q1102,plutoniuml,1539,Pu
103,1602,ununnilium,1427.0,Metal element with symbol Uuu and atomic numbe...,1602,http://www.wikidata.org/entity/Q1272,roentgenium,1602,Rg
104,1603,ununnubium,1427.0,Metal element with symbol Uub and atomic numbe...,1603,http://www.wikidata.org/entity/Q1301,nihonium,1603,Nh
105,1604,ununnunium,1427.0,Metal element with symbol Uun and atomic numbe...,1604,http://www.wikidata.org/entity/Q1278,copernicium,1604,Cn


# Sync GeoKB

We need to figure out what we already have in the GeoKB instance we are working against and bring in new items/claims as needed.

In [63]:
# GeoKB feedstock
q_natural_material = 'Q19'

# Property reference by label
geokb_properties = sparql_query(
    endpoint=os.environ["SPARQL_ENDPOINT_URL"],
    query=property_query,
    output='dict'
)

ref_geokb_properties = {}
for p in geokb_properties:
    ref_geokb_properties[p['propertyLabel']] = p['property'].split('/')[-1]

ref_geokb_properties

{'instance of': 'P1',
 'subclass of': 'P2',
 'reference item': 'P3',
 'reference url': 'P4',
 'reference statement': 'P5',
 'coordinate location': 'P6',
 'publication date': 'P7',
 'subject matter': 'P8',
 'ranking': 'P9',
 'ISO 3166-1 alpha-2 code': 'P10',
 'located in the administrative territorial entity': 'P11',
 'ISO 3166-2 code': 'P12',
 'FIPS 5-2 alpha code (US states)': 'P13',
 'FIPS 5-2 numeric code (US states)': 'P14',
 'corresponding wikidata property': 'P15',
 'related wikidata item': 'P16',
 'element symbol': 'P17',
 'SEDAR Identifier': 'P18',
 'MRDS commodity code': 'P19',
 'USGS Thesaurus ID': 'P20'}

## Item Classification

Each item we create here needs to have at least one instance of claim that classifies it into our GeoKB model. At some point, we need to work out a more elegant solution than I'm using here, but we have to start with something that gives us our basic classifiers.

In [35]:
def property_value_query(property, value):
    query = """
    PREFIX wd: <https://geokb.wikibase.cloud/entity/>
    PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>

    SELECT ?item ?itemLabel ?itemDescription 
    WHERE 
    {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    ?item wdt:%s wd:%s.
    }
    """ % (property, value)

    return query

### Basic Classifiers

I've organized a few high-level classifiers under the concept of "natural material." We'll be using `chemical element`, `chemical element group`, and `mineral commodity` in this case. We get the current identifiers for those classifiers and create a simple key/value structure to work with.

In [51]:
geokb_natural_materials = sparql_query(
    endpoint=os.environ['SPARQL_ENDPOINT_URL'],
    query=property_value_query(ref_geokb_properties['subclass of'], q_natural_material),
    output='dict'
)
ref_item_classification = {}
for item in geokb_natural_materials:
    ref_item_classification[item['itemLabel']] = item['item'].split('/')[-1]

ref_item_classification

{'mineral': 'Q24',
 'chemical element': 'Q280',
 'chemical element group': 'Q281',
 'mineral commodity': 'Q406'}

### Chemical Element Groups

There are several ways of classifying chemical elements, and we'll explore what makes best sense for our use cases over time. We won't need to get terribly in depth on our end as long as we maintain linkage to a source like MinDat where we can exploit different ways of classifying and characterizing mineral items in our system. Here, we bring in the USGS Thesaurus classification into chemical element groups as a simple start that we can explore further.

In [52]:
geokb_chem_elem_groups = sparql_query(
    endpoint=os.environ['SPARQL_ENDPOINT_URL'],
    query=property_value_query(ref_geokb_properties['instance of'], ref_item_classification['chemical element group']),
    output='dict'
)

ref_chemical_element_groups = {}
for item in geokb_chem_elem_groups:
    ref_chemical_element_groups[item['itemLabel']] = item['item'].split('/')[-1]

ref_chemical_element_groups

{'actinide series elements': 'Q282',
 'alkali metal elements': 'Q283',
 'alkaline earth elements': 'Q284',
 'halogen elements': 'Q285',
 'lanthanide series elements': 'Q286',
 'metal elements': 'Q287',
 'noble gas elements': 'Q288',
 'nonmetal elements': 'Q289',
 'rare earth elements': 'Q290',
 'platinum-group elements': 'Q291'}

In [44]:
# Check for any missing chemical element groups
# Revisit this in the workflow
chem_elem_groups[~chem_elem_groups['name'].isin(ref_chemical_element_groups.keys())]

,code,name,parent,scope


## Mineral Elements and Commodity Classification

The main thing we are trying to organize into the GeoKB at this point is a set of items representing mineral commodities. We have a reference source for the main concepts we find in our own data on chemical elements (that are minerals), some of which are also commodities, and additional commodity concepts as presented in an MRDS codebase. These are likely not the best foundational sources for a master set of commodity concepts in our GeoKB. We need to add in other classifying characteristics like whether the mineral is on the official 2022 list of critical minerals for the U.S. We'll revisit this reference source over time.

In [75]:
chem_elems_wd_mapping

,code,name,parent,scope,code_str,item,itemLabel,tcode,chem_symbol
0,1385,actinium,1427.0,Rare earth (actinide series) element with symb...,1385,http://www.wikidata.org/entity/Q1121,actinium,1385,Ac
1,1390,aluminum,1427.0,Metalloid element with symbol Al and atomic nu...,1390,http://www.wikidata.org/entity/Q663,aluminium,1390,Al
2,1392,americium,1427.0,Rare earth (actinide series) element with symb...,1392,http://www.wikidata.org/entity/Q1872,americium,1392,Am
3,1393,antimony,1427.0,Metalloid element with symbol Sb and atomic nu...,1393,http://www.wikidata.org/entity/Q1099,antimony,1393,Sb
4,1395,argon,1427.0,Noble gas element with symbol Ar and atomic nu...,1395,http://www.wikidata.org/entity/Q696,argon,1395,Ar
...,...,...,...,...,...,...,...,...,...
108,1613,xenon,1427.0,Noble gas element with symbol Xe and atomic nu...,1613,http://www.wikidata.org/entity/Q1106,xenon,1613,Xe
109,1616,ytterbium,1427.0,Rare earth (lanthanide series) element with sy...,1616,http://www.wikidata.org/entity/Q1855,ytterbium,1616,Yb
110,1617,yttrium,1427.0,Metal element with symbol Y and atomic number ...,1617,http://www.wikidata.org/entity/Q941,yttrium,1617,Y
111,1618,zinc,1427.0,Metal element with symbol Zn and atomic number...,1618,http://www.wikidata.org/entity/Q758,zinc,1618,Zn


In [93]:
query_missing_tcode = """
PREFIX wd: <https://geokb.wikibase.cloud/entity/>
PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>

SELECT DISTINCT ?item ?itemLabel 
WHERE {
    ?item  wdt:P1  wd:Q280 . 
    FILTER NOT EXISTS { ?item wdt:P20 [] } 
    SERVICE wikibase:label { bd:serviceParam  wikibase:language  "en" . } 
}
"""

missing_tcodes = sparql_query(
    endpoint=os.environ["SPARQL_ENDPOINT_URL"],
    query=query_missing_tcode,
    output='dataframe'
)

In [92]:
if not missing_tcodes.empty:
    df_tcode_map = pd.merge(
        left=missing_tcodes,
        right=chem_elems_wd_mapping[["name","code_str"]],
        how="left",
        left_on="itemLabel",
        right_on="name"
    )

    for index, row in df_tcode_map.iterrows():
        claim = ExternalID(
            prop_nr=ref_geokb_properties['USGS Thesaurus ID'],
            value=row.code_str
        )
        item = wbi.item.get(row['item'].split('/')[-1])
        item.add_claims(claims=claim)
        item.write()

In [97]:
query_missing_symbol = """
PREFIX wd: <https://geokb.wikibase.cloud/entity/>
PREFIX wdt: <https://geokb.wikibase.cloud/prop/direct/>

SELECT DISTINCT ?item ?itemLabel 
WHERE {
    ?item  wdt:P1  wd:Q280 . 
    FILTER NOT EXISTS { ?item wdt:P17 [] } 
    SERVICE wikibase:label { bd:serviceParam  wikibase:language  "en" . } 
}
"""

missing_symbols = sparql_query(
    endpoint=os.environ["SPARQL_ENDPOINT_URL"],
    query=query_missing_symbol,
    output='dataframe'
)

In [98]:
missing_symbols

""


In [ ]:
mrds_commodities = pd.read_csv(mrds_commodities_url, sep='\t')

In [ ]:
def cleaner_name(name):
    if ',' in name:
        name_parts = name.split(',')
        clean_name = " ".join([name_parts[1], name_parts[0]])
    else:
        clean_name = name

    return clean_name.lower()

mrds_commodities["cleaner_name"] = mrds_commodities["name"].apply(cleaner_name)

In [ ]:
minerals_commodities = pd.merge(
    left=mrds_commodities,
    right=chemical_elements[["item","itemLabel"]],
    how='left',
    left_on='cleaner_name',
    right_on='itemLabel'
)
minerals_commodities

In [ ]:
ref_mrds_commodity_list = URL(
    prop_nr=reference_url,
    value='https://mrdata.usgs.gov/mrds/commodity-codes.html'
)

claim_instance_of_commodity = Item(
    prop_nr=instance_of,
    value=mineral_commodity,
    references=[[ref_mrds_commodity_list]]
)

for index, row in minerals_commodities.iterrows():
    claim_commodity_code = ExternalID(
        prop_nr=mrds_commodity_code,
        value=row.code,
        references=[[ref_mrds_commodity_list]]
    )

    claims = [
        claim_instance_of_commodity,
        claim_commodity_code
    ]

    if isinstance(row['item'], str):
        qid = row['item'].split("/")[-1]
        item = wbi.item.get(qid)
        item.add_claims(claims)
        item.write()
    else:
        result = add_item(
            label=row.cleaner_name,
            description='a mineral-related commodity that is not a mineral element',
            aliases=row['name'],
            claims=claims
        )
        print(result)